# Transfer Learning for Skin Cancer Classification

In this notebook, we will demonstrate how to use transfer learning with pretrained models such as ResNet50 and EfficientNet for skin cancer classification using the HAM10000 dataset.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Load the dataset
data_path = '../data/GroundTruth.csv'
data = pd.read_csv(data_path)
data.head()

In [ ]:
# Prepare data for training and validation
X = data['image'].values
y = data[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Create the model using ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(7, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data,
    directory='../data/images/',
    x_col='image',
    y_col=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='multi_output'
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=data,
    directory='../data/images/',
    x_col='image',
    y_col=['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='multi_output'
)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[EarlyStopping(patience=3), ModelCheckpoint('best_model.h5', save_best_only=True)]
)

## Conclusion

In this notebook, we demonstrated how to implement transfer learning using pretrained models for skin cancer classification. We can further explore using EfficientNet and ensemble methods in subsequent notebooks.